In [31]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ofdm import *
from mod_16_qam import *
from fading import *
from gauss import *
from estimate import *
from equalize import *
import random

# 1 Mô Phỏng

In [32]:
x_time = 14 # 0-13
y_freq = 3276 # 0-3275
z_mod = 4

# đọc dữ liệu từ file Excel
df = pd.read_excel('data/l1_mini_project_ce_eq_dmrs_data.xlsx', sheet_name='Sheet1', header=None)
arr_r = df[0].apply(lambda x: complex(str(x).replace('i', 'j').replace(' ', ''))).to_numpy()

Tạo chuỗi bit ngẫu nhiêu (giá trị 0 và 1) có kích thước 3276 × 13 × 4

In [33]:
bits = ''.join(str(random.randint(0, 1)) for _ in range((x_time-1)*y_freq*z_mod))


Điều chế 16-QAM -> 3276x13 ký hiệu

In [34]:
symbols_16qam  = bits_to_symbols_16QAM(bits)

Ánh xạ vào lưới tài nguyên :trên xuống dưới, trừ trái sang phải:

In [35]:
grid = np.zeros((y_freq, x_time), dtype=complex)

for i in range(y_freq):
    for j in range(x_time):
        if j < 3:
            grid[i, j] = symbols_16qam[j*y_freq + i]
        elif j==3:
            grid[i, j] = arr_r[i]
        else:
            grid[i, j] = symbols_16qam[(j-1)*y_freq + i]

Từ grid -> ánh xạ sang OFDM

In [36]:
# ofdm cần thêm CP -> tín hiệu miền thời gian 61440 mẫu -> mà mới có 3276 * 14 = 61376 -> padding thêm
ofdm_symbol = ofdm_single_symbol(grid[:, 0], 4096, 288)

ofdm = np.zeros((4384, x_time), dtype=complex)
for i in range(x_time):
    ofdm[:, i] = ofdm_single_symbol(grid[:, i], 4096, 288)
    
# Ghép tất cả OFDM symbol thành 1 chuỗi dài 61376 mẫu
ofdm_signal = ofdm.flatten(order='F')  # flatten theo cột (trước sau)
# Padding thêm 64 mẫu 0 để đủ 61440 mẫu
padding = np.zeros(64, dtype=complex)
ofdm_signal = np.concatenate((ofdm_signal, padding))


Cho tín hiệu OFDM qua Fading channel  TDLB100-25

TDL: Tapped Delay Line – mô hình kênh đa đường có nhiều taps
B: Profile B (mức phản xạ trung bình)
100ns: Delay spread
25Hz: Doppler shift (UE di chuyển, tốc độ ~3km/h)

In [37]:
fs = 30.72e6  # sampling rate (Hz) cho 5G LTE BW = 20MHz (chuẩn)
channel_h = generate_tdlb_channel(fs)
faded_signal = apply_fading_channel(ofdm_signal, channel_h)
#faded_signal = ofdm_signal

Tiếp tục cho tín hiệu đi qua nhiễu Gauss có SNR 20dB


In [38]:
rx_signal = add_awgn(faded_signal, snr_db=20)

Giải điều chế OFDM

In [39]:
#rx_grid = ofdm_demodulate(rx_signal, fft_size=4096, cp_len=288, num_symbols=14)
rx_grid = ofdm_demodulate(rx_signal, fft_size=4096, cp_len=288, num_symbols=14)

# Cắt lại phần tương ứng với 3276 subcarriers (đầu tiên)
rx_grid = rx_grid[:3276, :]  # Kích thước (3276 x 14)

Giải điều chế 16QAM hardbit

In [40]:
data_cols = [i for i in range(14) if i != 3]  # loại bỏ cột 3 (DMRS)
rx_bits_all = []

for col in data_cols:
    rx_symbols = rx_grid[:, col]
    bits_col = qam16_demod_hard(rx_symbols)
    rx_bits_all.extend(bits_col)

print("Tổng số bit sau giải điều chế:", len(rx_bits_all))  # sẽ là 3276 * 13 * 4 = 170352


Tổng số bit sau giải điều chế: 170352


So sánh sai lệch dữ dữ liệu 16QAM giữa TX & RX theo MSE

In [41]:

bits_rx = np.array([int(b) for b in bits], dtype=int)
bits_tx = np.array([int(b) for b in rx_bits_all], dtype=int)

mse_bits = np.mean((bits_rx - bits_tx) ** 2)
print("MSE giữa bit TX và RX:", mse_bits)

MSE giữa bit TX và RX: 0.45829811214426597


So sánh và tính toán sai lệch bit (BER) giữa đầu phát và đầu thu.

In [42]:
ber = np.mean(bits_tx != bits_rx)
print("BER:", ber)

BER: 0.45829811214426597


# 2 ƯỚC LƯỢNG

Uớc lượng kênh truyền bằng thuật toán LS. 


In [43]:
#https://tailieusinhvien.net/phan-tich-so-sanh-ky-thuat-uoc-luong-kenh-ls-va-mmse-trong-he-thong-ofdm-duoi-nuoc-7436/
df = pd.read_excel('data/l1_mini_project_ce_eq_dmrs_data.xlsx', sheet_name='Sheet1', header=None)
X = df[0].apply(lambda x: complex(str(x).replace('i', 'j').replace(' ', ''))).to_numpy()

Y = rx_grid[:, 3]  # cột DMRS

# X= H.x +n -> tìm H sao cho ||Y-Hx||^2 min; H = X^-1 .Y
h_ls = estimate_channel_ls(X, Y)

Cân bằng kênh truyền bằng thuật toán ZF. 


In [44]:
rx_grid_equalized = equalize_zf(rx_grid, h_ls)
# Giải điều chế 16-QAM và tính BER/MSE
rx_bits_all_zf = []
data_cols = [i for i in range(14) if i != 3]
for col in data_cols:
    rx_symbols = rx_grid_equalized[:, col]
    bits_col = qam16_demod_hard(rx_symbols)
    rx_bits_all_zf.extend(bits_col)

bits_rx = np.array([int(b) for b in bits], dtype=int)
bits_tx_zf = np.array([int(b) for b in rx_bits_all_zf], dtype=int)
mse_zf = np.mean((bits_rx - bits_tx_zf) ** 2)
ber_zf = np.mean(bits_rx != bits_tx_zf)

print(f"MSE (ZF): {mse_zf:.6f}")
print(f"BER (ZF): {ber_zf:.6f}")

MSE (ZF): 0.012668
BER (ZF): 0.012668


Thay đổi giá trị SNR từ 0 đến 30, khoảng cách 2dB. Vẽ biểu đồ MSE và biểu đồ BER.


So sánh MSE và BLER giữa có và không có sử dụng ước lượng/cân bằng kênh truyền. 


# 3 Phát triển và đánh giá thuật toán ước lượng kênh truyền nâng cao.

Thay thế thuật toán LS bằng thuật toán ước lượng kênh truyền nâng cao MMSE, sử 
dụng bộ cân bằng MMSE và so sánh biểu đồ MSE, BER với thuật toán LS/ZF.

In [45]:
h_mmse = estimate_channel_mmse(X, Y, snr_db=20)  # SNR = 20 dB

# Áp dụng cân bằng MMSE thay vì ZF
snr_db = 20  # Giả định SNR = 20 dB như trong notebook
rx_grid_equalized = np.zeros_like(rx_grid, dtype=complex)
for col in range(x_time):
    if col != 3:  # Bỏ qua cột DMRS
        rx_grid_equalized[:, col] = equalize_mmse(rx_grid[:, col], h_mmse, snr_db)
    else:
        rx_grid_equalized[:, col] = rx_grid[:, col]

# Giải điều chế 16-QAM và tính BER/MSE
rx_bits_all_mmse = []
data_cols = [i for i in range(14) if i != 3]
for col in data_cols:
    rx_symbols = rx_grid_equalized[:, col]
    bits_col = qam16_demod_hard(rx_symbols)
    rx_bits_all_mmse.extend(bits_col)

bits_rx = np.array([int(b) for b in bits], dtype=int)
bits_tx_mmse = np.array([int(b) for b in rx_bits_all_mmse], dtype=int)
mse_mmse = np.mean((bits_rx - bits_tx_mmse) ** 2)
ber_mmse = np.mean(bits_rx != bits_tx_mmse)

print(f"MSE (MMSE): {mse_mmse:.6f}")
print(f"BER (MMSE): {ber_mmse:.6f}")


MSE (MMSE): 0.013026
BER (MMSE): 0.013026


Thực hiện đánh giá lại các phương pháp với số lượng mẫu là 10000 lưới tài nguyên, 
bổ sung thêm các kiểu điều chế QPSK và 64QAM.